<a href="https://colab.research.google.com/github/Tanmay-boop-hash/24B0394_AIC/blob/main/RAG_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the PyMuPDF library
(it is a high performance library by python used for manipulation, data extraction, analysis of PDF documents)

In [2]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 85.0 MB/s eta 0:00:00


Uploading the PDF file

In [3]:
from google.colab import files
uploaded = files.upload()

Saving PP-001-Space.pdf to PP-001-Space.pdf


Read and chunk the pdf

In [4]:
import fitz

def extract_pdf_text_chunks(file_path, max_chunk_length=500):
  doc = fitz.open(file_path)
  chunks = []
  current_chunk = ""

  for page in doc:
    text = page.get_text()
    for para in text.split("\n"):
      if len(current_chunk) + len(para) < max_chunk_length:
        current_chunk += para.strip() + " "
      else:
        chunks.append(current_chunk.strip())
        current_chunk = para.strip() + " "

  if current_chunk:
    chunks.append(current_chunk.strip())

  return chunks

chunks = extract_pdf_text_chunks("PP-001-Space.pdf")
chunk_texts = chunks.copy()

# print a few example chunks
for i, chunk in enumerate(chunks[:5]):
  print(f"\n---Chunk {i+1} ---\n {chunk}")


---Chunk 1 ---
 Dr. A.P.J. Abdul Kalam The Future of Space Exploration and Human Development The PARDEE PAPERS  /  No. 1  /  August 2008  The Future of Space Exploration and Human Development  Dr. A.P.J. Abdul Kalam The PARDEE PAPERS  /  No. 1  /  August 2008  The Pardee Papers series features working papers by Pardee Center Fellows and other invited authors. Papers in this series explore current and future challenges by anticipating the pathways to human progress, human development, and human

---Chunk 2 ---
 well-being. This series includes papers on a wide range of topics, but with a special emphasis on interdisciplinary perspectives and a development orientation. Series Editor: Professor Adil Najam The Frederick S. Pardee Center for the Study of the Longer-Range Future at Boston University convenes and conducts interdisciplinary, policy-relevant, and future-oriented research that can contribute to long-term improvements in the

---Chunk 3 ---
 human condition. Through its programs

Installing other required libraries

In [5]:
!pip install faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

Generating embeddings from chunks of text

In [6]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Loading a pretrained embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Converting each chunk into a vector
embeddings = model.encode(chunks, show_progress_bar=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Creating FAISS Index

In [7]:
import faiss

dimension = embeddings.shape[1]  # Length of each embedding
index = faiss.IndexFlatL2(dimension)  # L2 = Euclidean distance
index.add(np.array(embeddings)) # add all embeddings to the index

# Save the chunks along with embeddings for later use


Sample chunk printing test

In [8]:
print("Number of embeddings:", len(embeddings))
print("Number of chunks:", len(chunk_texts))
i=0
print("\nSample Embedding Length:", len(embeddings[i]))
print("Sample Chunk:", chunk_texts[i])

Number of embeddings: 66
Number of chunks: 66

Sample Embedding Length: 384
Sample Chunk: Dr. A.P.J. Abdul Kalam The Future of Space Exploration and Human Development The PARDEE PAPERS  /  No. 1  /  August 2008  The Future of Space Exploration and Human Development  Dr. A.P.J. Abdul Kalam The PARDEE PAPERS  /  No. 1  /  August 2008  The Pardee Papers series features working papers by Pardee Center Fellows and other invited authors. Papers in this series explore current and future challenges by anticipating the pathways to human progress, human development, and human


Defining a function for getting the top k relevant chunks for a question

In [9]:
def get_top_k_chunks(question, k=3):
  question_embedding = model.encode([question])   # Convert question to embedding
  distances, indices = index.search(np.array(question_embedding), k)   # Search FAISS for similar chunks
  results = [chunk_texts[i] for i in indices[0]]  # Get the top-k chunks
  return results

First loading a basic model like distilbert and testing it on our pdf file

In [10]:
!pip install transformers
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


Generating the answer from the relevant chunks after a question is asked

In [11]:
def answer_question(question):
  top_chunks = get_top_k_chunks(question, k=3)
  context = " ".join(top_chunks)
  result = qa_pipeline({'question':question, 'context':context})
  return result['answer']

Sample question and relevant chunks for it shown

In [12]:
question = "What is India's vision for space by 2050?"
top_chunks = get_top_k_chunks(question, k=5)
for i, chunk in enumerate(top_chunks):
    print(f"\n--- Chunk {i+1} ---\n {chunk}")


--- Chunk 1 ---
 technology including space. It is critical to continue researching how space technology can enhance further the quality of life of world citizens, while examining many innovative ideas for space exploration, including exciting aspects like space colonies, search for extra terrestrial intelligence and plan- etary exploration. We have to understand our planet more completely, and evolve and adapt our life to its environment and ecology. Indian Space Visionary and the Indian Space Programme

--- Chunk 2 ---
 space missions. India is now working on its second space vision. It is quite possible that India will make an important contribution for the future of exploration with space missions to the Moon and Mars founded on space industrialization. Space Research as a Technology Generator Space research is truly inter-disciplinary and has enabled true innovations at the intersection of multiple areas of science and engineering. It has been consistently aiming at the “im- poss

Getting the Grog API key

In [13]:
GROQ_API_KEY = "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"

In [14]:
!pip install requests

Defining a function to call Groq

In [15]:
import requests

def call_groq_llm(prompt, model="llama3-8b-8192", api_key="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"):
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant who answers questions based on provided context."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7
    }

    response = requests.post(url, headers=headers, json=data)
    result = response.json()
    return result['choices'][0]['message']['content']


Building the final prompt and generating the answer

In [16]:
def answer_with_groq(question, context_chunks, api_key):
  context = "\n\n".join(context_chunks)
  prompt = f"""Use the folowing context to answer the question:
  Context:{context}
  Question:{question}
  """
  return call_groq_llm(prompt, api_key=api_key)

In [40]:
def info_extraction_agent(chunks, api_key):
    context = "\n\n".join(chunks)
    prompt = f"Extract key facts, figures, and concepts from the following text:\n\n{context}"
    return call_groq_llm(prompt, api_key=api_key)

def synthesis_agent(facts_text, api_key):
    prompt = f"Organize the following information into a concise summary or logical structure:\n\n{facts_text}"
    return call_groq_llm(prompt, api_key=api_key)

def query_agent(question, summary_text, api_key):
    prompt = f"""Use the following information to answer the user's question.
              Information:{summary_text}
              Question:{question}"""
    return call_groq_llm(prompt, api_key=api_key)


In [41]:
def coordinator_pipeline(question, api_key, k=5):
    # Step 1: Retrieve chunks
    chunks = get_top_k_chunks(question, k)

    # Step 2: Extract key facts
    facts = info_extraction_agent(chunks, api_key)

    # Step 3: Organize into a summary
    summary = synthesis_agent(facts, api_key)

    # Step 4: Answer the question
    answer = query_agent(question, summary, api_key)

    return answer


History-Aware responses

In [47]:
def rewrite_question(follow_up_question, chat_history, api_key):
    history_text = ""
    for turn in chat_history[-2:]:  # Use last 2 turns
        history_text += f"User: {turn['question']}\nAssistant: {turn['answer']}\n"

    prompt = f"""Rewrite the following follow-up question as a complete standalone question, using the prior conversation.
                 Conversation:{history_text}
                 Follow-up question: {follow_up_question}
                 Standalone question:"""

    return call_groq_llm(prompt, api_key=api_key).strip()


In [48]:
chat_history = []

def smart_agent_chat(follow_up_question, api_key):
    # Rewrite the vague question using chat history
    if chat_history:
        rewritten = rewrite_question(follow_up_question, chat_history, api_key)
    else:
        rewritten = follow_up_question

    # Run coordinator pipeline on rewritten question
    answer = coordinator_pipeline(rewritten, api_key)

    # Save to history
    chat_history.append({"question": follow_up_question, "answer": answer})

    return rewritten, answer


In [49]:
# First question (clear one)
q1 = "What are the main challenges in space missions?"
rew1, a1 = smart_agent_chat(q1, api_key)
print(f"\nQ: {rew1}\nA: {a1}")

# Follow-up (vague)
q2 = "And what are the solutions?"
rew2, a2 = smart_agent_chat(q2, api_key)
print(f"\nQ: {rew2}\nA: {a2}")



Q: What are the main challenges in space missions?
A: According to the provided information, the main challenges in space missions are:

1. Attracting students to pursue careers in space research and technology
2. Ensuring a steady stream of young minds embracing space research
3. Maintaining peace in space by avoiding terrestrial geo-political conflicts

These challenges are addressed in Section II of the logical structure, specifically under the "Challenges" heading.

Q: What are the solutions to the main challenges in space missions, such as attracting students, ensuring a steady stream of young minds, and maintaining peace in space?
A: According to the provided information, the solutions to the main challenges in space missions, such as attracting students, ensuring a steady stream of young minds, and maintaining peace in space, are:

1. Implementing a global strategic plan for space industrialization over the next 50 years, with a goal of having youngsters fully accomplished in e

In [43]:
api_key = "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"
question = "What role does the Moon play in future space missions?"
top_chunks = get_top_k_chunks(question, k=5)
answer = coordinator_pipeline(question, api_key)
print("Answer:", answer)

Answer: According to the provided information, the Moon plays a crucial role in future space missions as a potential hub for interplanetary travel, interplanetary communications, and space construction sites. Its unique environment makes it an ideal location for:

1. Clear sky for waves of all frequencies, making it suitable for interplanetary communication systems.
2. Shielding from Earth's continuous radio emissions, reducing interference and providing a quiet environment for communication.
3. Potential source of construction materials, such as mining in planets or asteroids.
4. Ideal environment for large-area solar converters, which could be used to generate power for space missions.

The Moon's unique characteristics make it an attractive location for future space missions, and it is expected to play a key role in India's second space vision, which may involve missions to the Moon and Mars.


Mini Knowledge graph : using the model to extract facts from each chunk in form of triplets

In [26]:
def extract_facts_from_chunk(chunk, api_key):
    prompt = f"""Extract factual relationships as (Subject, Predicate, Object) triples from this text:
              {chunk}
              Output only the list of triples."""
    response = call_groq_llm(prompt, api_key=api_key)
    return response


In [29]:
for chunk in chunk_texts[20:25]:
    facts = extract_facts_from_chunk(chunk, api_key)
    print(f"\nFacts from chunk:\n{facts}")


Facts from chunk:
Here are the factual relationships extracted as (Subject, Predicate, Object) triples:

• The Future of Space Exploration and Human Development, is related to, perennial supply of renewable energy from space
• Space research, has resulted in, designing innovative products
• Space research, has resulted in, cardiac stent
• Space research, has resulted in, heart pacemaker
• Space research, has contributed to, revolution in communication
• Space research, has contributed to, tele-presence
• Space research, has contributed to, infotainment
• Space research, has contributed to, integrated picture of earth and its resources

Facts from chunk:
Here are the factual relationships as (Subject, Predicate, Object) triples:

* The world population, is, 6.6 billion
* The world population, is projected to be, more than 9 billion
* The world population, is projected to be, by 2050
* The critical issues, arise from, population growth
* Population growth, causes, shortage of energy
* P